In [ ]:
# @title
# %%capture
# # Installs Unsloth, Xformers (Flash Attention) and all other packages!
# !pip install unsloth
# # Get latest Unsloth
# !pip install --upgrade --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
# @title
# from unsloth import FastLanguageModel
# import torch
# max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
# dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
# load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# # 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
#     "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
#     "unsloth/llama-3-8b-Instruct-bnb-4bit",
#     "unsloth/llama-3-70b-bnb-4bit",
#     "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
#     "unsloth/Phi-3-medium-4k-instruct",
#     "unsloth/Qwen2-0.5b-bnb-4bit",           # Qwen2 2x faster!
#     "unsloth/Qwen2-1.5b-bnb-4bit",
#     "unsloth/Qwen2-7b-bnb-4bit",
#     "unsloth/Qwen2-72b-bnb-4bit",
#     "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
# ] # Try more models at https://huggingface.co/unsloth!

# model, tokenizer = FastLanguageModel.from_pretrained(
#     # model_name = "unsloth/Qwen2-7B", # Reminder we support ANY Hugging Face model!
#     model_name = "unsloth/Qwen2.5-7B-instruct",
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
#     # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
# )

In [ ]:
# @title
# model = FastLanguageModel.get_peft_model(
#     model,
#     r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
#     target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
#                       "gate_proj", "up_proj", "down_proj",],
#     lora_alpha = 16,
#     lora_dropout = 0, # Supports any, but = 0 is optimized
#     bias = "none",    # Supports any, but = "none" is optimized
#     # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
#     use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
#     random_state = 3407,
#     use_rslora = False,  # We support rank stabilized LoRA
#     loftq_config = None, # And LoftQ
# )

In [ ]:
# @title
# alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# {}"""

# text = "Although, on average, climate change may have a negative impact on yields, the projected yields by 2050 are still higher than current yields for most commodities. In the case of maize, for instance, under Scenario 4, climate may reduce the potential yields by almost 20%, however the maize yields are still expected to be about 50% higher than they were in 2005. The two scenarios that use inputs from the DSSAT crop model (Scenario 3 (IPSL/DSSAT) and 4 (Hadley/DSSAT)) result in higher world prices for maize, wheat and rice than do the two scenarios using the LPJmL crop model (Scenario 1 (IPSL/LPJmL) and 2 (Hadley/LPJmL))."
# sdg_true = 12

# template = """Given the following input text, between triple quotes, with its associated classification label with respect to one of the Sustainable Development Goals (SDGs), further classify the text with respect to the three labels defined below:

#   - "positive": the text implies or explicitly states that what is affirmed or described leads to a significant advancement in favor of the goal indicated by the SDG under which it was previously classified.

#   - "negative": the text implies or explicitly states that what is affirmed or described leads to a significant advancement against the goal indicated by the SDG under which it was previously classified.

#   - "neutral": the text does not imply or explicitly state that what is affirmed or described leads to a significant advancement either in favor or against the goal indicated by the SDG under which it was previously classified.

# Return the result in JSON format with 4 keys:

#   - "label": The assigned label for the input text [this key/value pair is mandatory].

#   - "explanation_1": A description of the reasoning behind the classification decision [this key/value pair is mandatory].

#   - "explanation_2": Additional description of the reasoning behind the classification decision [this key/value pair is optional].

#   - "explanation_3": Another additional description of the reasoning behind the classification decision [this key/value pair is optional].
# """

# input = """INPUT TEXT: \""""+text+"""\"
# SDG_CLASSIFICATION: "SDG-"""+str(sdg_true)+"""\"
# """

# FastLanguageModel.for_inference(model) # Unsloth has 2x faster inference!
# inputs = tokenizer(
# [
#     alpaca_prompt.format(
#         template, #"Continue the fibonnaci sequence.", # instruction
#         input, #"1, 1, 2, 3, 5, 8", # input
#         "", # output - leave this blank for generation!
#     )
# ], return_tensors = "pt").to("cuda")

# prompt_length = len(alpaca_prompt)+inputs['input_ids'].shape[1]

# outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)
# tokenizer.batch_decode(outputs)

In [ ]:
# @title
# prompt_length = inputs['input_ids'].shape[1]

# tokenizer.batch_decode(outputs)[0][prompt_length:]

In [ ]:
import pandas as pd
import pickle

In [ ]:
experiment_name = "2024.10.10-03-AnalisiSentimentConQwen"

In [ ]:
table_name = 'vinc_Qwen_sentiment_processed_ids'

In [ ]:
# model_name = "Qwen/Qwen1.5-7B"
model_name = "unsloth/Qwen2.5-7B-instruct"

In [ ]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"

input_data_path = base_path + "/../AnalisiTesti/data/output/full_data___6400_texts___sentiment_sdgwashing_sdgsmearing.xlsx"

output_data_path = base_path + "/../AnalisiTesti/data/output/" + experiment_name
log_filename = base_path + "/../AnalisiTesti/logs/" + experiment_name + ".log"

drive.mount(mount_point, force_remount=True)

Mounted at /content/gdrive


In [ ]:
with open(base_path+"/../AnalisiTesti/cred.txt", 'r') as file:
    lines = file.readlines()

my_aws_access_key_id = lines[0].strip()
my_aws_secret_access_key = lines[1].strip()
my_aws_session_token = lines[2].strip()
huggingface_cli_token = lines[3].strip()

In [ ]:
# from IPython.display import clear_output
# huggingface_cli_token = input("INSERT YOUR huggingface-cli_token:")
# clear_output()

In [ ]:
!huggingface-cli login --token $huggingface_cli_token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
pip install -q boto3 --progress-bar off

In [ ]:
import boto3

# Inizializza il client DynamoDB con le credenziali manuali
dynamodb = boto3.resource(
    'dynamodb',
    region_name='eu-west-1',
    aws_access_key_id=my_aws_access_key_id,
    aws_secret_access_key=my_aws_secret_access_key,
    aws_session_token=my_aws_session_token
)

table = dynamodb.Table(table_name)

In [ ]:
# Funzione per aggiungere un nuovo record
def add_record(table, text_id):
    # Dati del nuovo record
    item = {
        'text_id': text_id  # Colonna e valore da inserire
        # Aggiungi qui altre colonne se necessario, ad esempio:
        # 'another_column': 'some_value'
    }

    # Aggiungi il record nella tabella
    try:
        table.put_item(Item=item)
        print(f"Record aggiunto con successo: {item}")
    except Exception as e:
        print(f"Errore durante l'inserimento del record: {e}")

# Funzione per estrarre tutti i valori della colonna 'text_id'
def extract_all_processed_text_ids(table):
    try:
        # Scansione iniziale della tabella
        response = table.scan()
        items = response['Items']

        # Lista per contenere i valori 'text_id'
        text_id_list = [item['text_id'] for item in items]

        # Gestione della paginazione (se ci sono più pagine di dati)
        while 'LastEvaluatedKey' in response:
            response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
            items = response['Items']
            text_id_list.extend([item['text_id'] for item in items])

        return text_id_list

    except Exception as e:
        print(f"Errore durante l'estrazione: {e}")
        #return []
        raise(e)

In [ ]:
processed_text_ids = extract_all_processed_text_ids(table)
print(f"Sono gia' stati processati {len(processed_text_ids)} testi")

Sono gia' stati processati 1124 testi


In [ ]:
import pickle

In [ ]:
# @title
import pandas as pd
data = pd.read_excel(input_data_path)

In [ ]:
import glob

def update_data(data_sel):

  processed_texts_from_dynamodb = extract_all_processed_text_ids(table)

  processed_texts_from_dynamodb = [int(x) for x in processed_texts_from_dynamodb]

  #print(processed_texts_from_dynamodb)

  data = data_sel[~data_sel.text_id.isin(processed_texts_from_dynamodb)].copy() # DEEP COPY

  print("Amount of processed files: ", len(processed_texts_from_dynamodb))

  return data.sample(frac=1.00)

# data_sel = update_data(data_sel)
# data_sel.shape[0]

## MODELLO QUANTIZZATO (RICHIEDE UNA GPU CON PIU' DI 15 GB DI VRAM)

In [ ]:
# !pip install torchvision==0.19.1 # utile solo per kaggle!!! (vedi https://github.com/vllm-project/vllm/issues/5501)
# !pip install vllm

In [ ]:
# !pip install --upgrade transformers bitsandbytes accelerate datasets

In [ ]:
# import torch
# from vllm import LLM, SamplingParams
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# model_name = "Qwen/Qwen1.5-7B"

# tokenizer = AutoTokenizer.from_pretrained(model_name)

# compute_dtype = getattr(torch, "float16")

# bnb_config = BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_quant_type="nf4",
#         bnb_4bit_compute_dtype=compute_dtype,
#         bnb_4bit_use_double_quant=True,
# )
# model = AutoModelForCausalLM.from_pretrained(
#           model_name, quantization_config=bnb_config
# )

# sampling_params = SamplingParams(temperature=0.7, top_p=0.8, top_k=20, max_tokens=150)

# llm = LLM(model="Qwen/Qwen1.5-7B", dtype=torch.float16)

In [ ]:
# @title
# pip freeze

# absl-py==1.4.0
# accelerate==1.0.0
# aiohappyeyeballs==2.4.3
# aiohttp==3.10.9
# aiosignal==1.3.1
# alabaster==0.7.16
# albucore==0.0.16
# albumentations==1.4.15
# altair==4.2.2
# annotated-types==0.7.0
# anyio==3.7.1
# argon2-cffi==23.1.0
# argon2-cffi-bindings==21.2.0
# array_record==0.5.1
# arviz==0.19.0
# astropy==6.1.4
# astropy-iers-data==0.2024.10.7.0.32.46
# astunparse==1.6.3
# async-timeout==4.0.3
# atpublic==4.1.0
# attrs==24.2.0
# audioread==3.0.1
# autoawq==0.2.6
# autoawq_kernels==0.0.7
# autograd==1.7.0
# babel==2.16.0
# backcall==0.2.0
# beautifulsoup4==4.12.3
# bigframes==1.21.0
# bigquery-magics==0.4.0
# bitsandbytes==0.44.1
# bleach==6.1.0
# blinker==1.4
# blis==0.7.11
# blosc2==2.0.0
# bokeh==3.4.3
# boto3==1.35.37
# botocore==1.35.37
# Bottleneck==1.4.0
# bqplot==0.12.43
# branca==0.8.0
# build==1.2.2.post1
# CacheControl==0.14.0
# cachetools==5.5.0
# catalogue==2.0.10
# certifi==2024.8.30
# cffi==1.17.1
# chardet==5.2.0
# charset-normalizer==3.3.2
# chex==0.1.87
# clarabel==0.9.0
# click==8.1.7
# cloudpathlib==0.19.0
# cloudpickle==2.2.1
# cmake==3.30.4
# cmdstanpy==1.2.4
# colorcet==3.1.0
# colorlover==0.3.0
# colour==0.1.5
# community==1.0.0b1
# confection==0.1.5
# cons==0.4.6
# contextlib2==21.6.0
# contourpy==1.3.0
# cryptography==43.0.1
# cuda-python==12.2.1
# cudf-cu12 @ https://pypi.nvidia.com/cudf-cu12/cudf_cu12-24.6.1-cp310-cp310-manylinux_2_28_x86_64.whl#sha256=6fdd6fd412117503ad23dca75aabb5b536348d3af459b59920152be1a0af6f15
# cufflinks==0.17.3
# cupy-cuda12x==12.2.0
# cvxopt==1.3.2
# cvxpy==1.5.3
# cycler==0.12.1
# cymem==2.0.8
# Cython==3.0.11
# dask==2024.8.0
# datascience==0.17.6
# datasets==3.0.1
# db-dtypes==1.3.0
# dbus-python==1.2.18
# debugpy==1.6.6
# decorator==4.4.2
# defusedxml==0.7.1
# Deprecated==1.2.14
# dill==0.3.8
# diskcache==5.6.3
# distributed==2024.8.0
# distro==1.7.0
# dlib==19.24.2
# dm-tree==0.1.8
# docstring_parser==0.16
# docutils==0.18.1
# dopamine_rl==4.0.9
# duckdb==1.1.1
# earthengine-api==1.0.0
# easydict==1.13
# ecos==2.0.14
# editdistance==0.8.1
# eerepr==0.0.4
# einops==0.8.0
# en-core-web-sm @ https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl#sha256=86cc141f63942d4b2c5fcee06630fd6f904788d2f0ab005cce45aadb8fb73889
# entrypoints==0.4
# et-xmlfile==1.1.0
# etils==1.9.4
# etuples==0.3.9
# eval_type_backport==0.2.0
# exceptiongroup==1.2.2
# fastai==2.7.17
# fastapi==0.115.0
# fastcore==1.7.11
# fastdownload==0.0.7
# fastjsonschema==2.20.0
# fastprogress==1.0.3
# fastrlock==0.8.2
# filelock==3.16.1
# firebase-admin==6.5.0
# Flask==2.2.5
# flatbuffers==24.3.25
# flax==0.8.5
# folium==0.17.0
# fonttools==4.54.1
# frozendict==2.4.5
# frozenlist==1.4.1
# fsspec==2024.6.1
# future==1.0.0
# gast==0.6.0
# gcsfs==2024.6.1
# GDAL==3.6.4
# gdown==5.2.0
# geemap==0.34.5
# gensim==4.3.3
# geocoder==1.38.1
# geographiclib==2.0
# geopandas==1.0.1
# geopy==2.4.1
# gguf==0.10.0
# gin-config==0.5.0
# glob2==0.7
# google==2.0.3
# google-ai-generativelanguage==0.6.6
# google-api-core==2.19.2
# google-api-python-client==2.137.0
# google-auth==2.27.0
# google-auth-httplib2==0.2.0
# google-auth-oauthlib==1.2.1
# google-cloud-aiplatform==1.69.0
# google-cloud-bigquery==3.25.0
# google-cloud-bigquery-connection==1.15.5
# google-cloud-bigquery-storage==2.26.0
# google-cloud-bigtable==2.26.0
# google-cloud-core==2.4.1
# google-cloud-datastore==2.19.0
# google-cloud-firestore==2.16.1
# google-cloud-functions==1.16.5
# google-cloud-iam==2.15.2
# google-cloud-language==2.13.4
# google-cloud-pubsub==2.25.2
# google-cloud-resource-manager==1.12.5
# google-cloud-storage==2.8.0
# google-cloud-translate==3.15.5
# google-colab @ file:///colabtools/dist/google_colab-1.0.0.tar.gz#sha256=4ee0b0c952d6781a51c4eee286d5c303acd4cb43570df746a086832095268f49
# google-crc32c==1.6.0
# google-generativeai==0.7.2
# google-pasta==0.2.0
# google-resumable-media==2.7.2
# googleapis-common-protos==1.65.0
# googledrivedownloader==0.4
# graphviz==0.20.3
# greenlet==3.1.1
# grpc-google-iam-v1==0.13.1
# grpcio==1.64.1
# grpcio-status==1.48.2
# gspread==6.0.2
# gspread-dataframe==3.3.1
# gym==0.25.2
# gym-notices==0.0.8
# h11==0.14.0
# h5netcdf==1.4.0
# h5py==3.11.0
# holidays==0.57
# holoviews==1.19.1
# html5lib==1.1
# httpcore==1.0.6
# httpimport==1.4.0
# httplib2==0.22.0
# httptools==0.6.1
# httpx==0.27.2
# huggingface-hub==0.24.7
# humanize==4.10.0
# hyperopt==0.2.7
# ibis-framework==9.2.0
# idna==3.10
# imageio==2.35.1
# imageio-ffmpeg==0.5.1
# imagesize==1.4.1
# imbalanced-learn==0.12.4
# imgaug==0.4.0
# immutabledict==4.2.0
# importlib_metadata==8.4.0
# importlib_resources==6.4.5
# imutils==0.5.4
# inflect==7.4.0
# iniconfig==2.0.0
# intel-cmplr-lib-ur==2024.2.1
# intel-openmp==2024.2.1
# interegular==0.3.3
# ipyevents==2.0.2
# ipyfilechooser==0.6.0
# ipykernel==5.5.6
# ipyleaflet==0.19.2
# ipyparallel==8.8.0
# ipython==7.34.0
# ipython-genutils==0.2.0
# ipython-sql==0.5.0
# ipytree==0.2.2
# ipywidgets==7.7.1
# itsdangerous==2.2.0
# jax==0.4.33
# jax-cuda12-pjrt==0.4.33
# jax-cuda12-plugin==0.4.33
# jaxlib==0.4.33
# jeepney==0.7.1
# jellyfish==1.1.0
# jieba==0.42.1
# Jinja2==3.1.4
# jiter==0.6.1
# jmespath==1.0.1
# joblib==1.4.2
# jsonpickle==3.3.0
# jsonschema==4.23.0
# jsonschema-specifications==2023.12.1
# jupyter-client==6.1.12
# jupyter-console==6.1.0
# jupyter-leaflet==0.19.2
# jupyter-server==1.24.0
# jupyter_core==5.7.2
# jupyterlab_pygments==0.3.0
# jupyterlab_widgets==3.0.13
# kaggle==1.6.17
# kagglehub==0.3.1
# keras==3.4.1
# keyring==23.5.0
# kiwisolver==1.4.7
# langcodes==3.4.1
# language_data==1.2.0
# lark==1.2.2
# launchpadlib==1.10.16
# lazr.restfulclient==0.14.4
# lazr.uri==1.0.6
# lazy_loader==0.4
# libclang==18.1.1
# librosa==0.10.2.post1
# lightgbm==4.5.0
# linkify-it-py==2.0.3
# llvmlite==0.43.0
# lm-format-enforcer==0.10.6
# locket==1.0.0
# logical-unification==0.4.6
# lxml==4.9.4
# marisa-trie==1.2.0
# Markdown==3.7
# markdown-it-py==3.0.0
# MarkupSafe==2.1.5
# matplotlib==3.7.1
# matplotlib-inline==0.1.7
# matplotlib-venn==1.1.1
# mdit-py-plugins==0.4.2
# mdurl==0.1.2
# miniKanren==1.0.3
# missingno==0.5.2
# mistral_common==1.4.4
# mistune==0.8.4
# mizani==0.11.4
# mkl==2024.2.2
# ml-dtypes==0.4.1
# mlxtend==0.23.1
# more-itertools==10.5.0
# moviepy==1.0.3
# mpmath==1.3.0
# msgpack==1.0.8
# msgspec==0.18.6
# multidict==6.1.0
# multipledispatch==1.0.0
# multiprocess==0.70.16
# multitasking==0.0.11
# murmurhash==1.0.10
# music21==9.1.0
# namex==0.0.8
# natsort==8.4.0
# nbclassic==1.1.0
# nbclient==0.10.0
# nbconvert==6.5.4
# nbformat==5.10.4
# nest-asyncio==1.6.0
# networkx==3.3
# nibabel==5.2.1
# nltk==3.8.1
# notebook==6.5.5
# notebook_shim==0.2.4
# numba==0.60.0
# numexpr==2.10.1
# numpy==1.26.4
# nvidia-cublas-cu12==12.1.3.1
# nvidia-cuda-cupti-cu12==12.1.105
# nvidia-cuda-nvcc-cu12==12.6.77
# nvidia-cuda-nvrtc-cu12==12.1.105
# nvidia-cuda-runtime-cu12==12.1.105
# nvidia-cudnn-cu12==9.1.0.70
# nvidia-cufft-cu12==11.0.2.54
# nvidia-curand-cu12==10.3.2.106
# nvidia-cusolver-cu12==11.4.5.107
# nvidia-cusparse-cu12==12.1.0.106
# nvidia-ml-py==12.560.30
# nvidia-nccl-cu12==2.20.5
# nvidia-nvjitlink-cu12==12.6.77
# nvidia-nvtx-cu12==12.1.105
# nvtx==0.2.10
# oauth2client==4.1.3
# oauthlib==3.2.2
# openai==1.51.2
# opencv-contrib-python==4.10.0.84
# opencv-python==4.10.0.84
# opencv-python-headless==4.10.0.84
# openpyxl==3.1.5
# opentelemetry-api==1.27.0
# opentelemetry-sdk==1.27.0
# opentelemetry-semantic-conventions==0.48b0
# opt_einsum==3.4.0
# optax==0.2.3
# optree==0.13.0
# orbax-checkpoint==0.6.4
# osqp==0.6.7.post0
# outlines==0.0.46
# packaging==24.1
# pandas==2.2.2
# pandas-datareader==0.10.0
# pandas-gbq==0.23.2
# pandas-stubs==2.2.2.240909
# pandocfilters==1.5.1
# panel==1.4.5
# param==2.1.1
# parso==0.8.4
# parsy==2.1
# partd==1.4.2
# partial-json-parser==0.2.1.1.post4
# pathlib==1.0.1
# patsy==0.5.6
# peewee==3.17.6
# pexpect==4.9.0
# pickleshare==0.7.5
# pillow==10.4.0
# pip-tools==7.4.1
# platformdirs==4.3.6
# plotly==5.24.1
# plotnine==0.13.6
# pluggy==1.5.0
# polars==1.7.1
# pooch==1.8.2
# portpicker==1.5.2
# prefetch_generator==1.0.3
# preshed==3.0.9
# prettytable==3.11.0
# proglog==0.1.10
# progressbar2==4.5.0
# prometheus-fastapi-instrumentator==7.0.0
# prometheus_client==0.21.0
# promise==2.3
# prompt_toolkit==3.0.48
# prophet==1.1.6
# proto-plus==1.24.0
# protobuf==3.20.3
# psutil==5.9.5
# psycopg2==2.9.9
# ptyprocess==0.7.0
# py-cpuinfo==9.0.0
# py4j==0.10.9.7
# pyairports==2.1.1
# pyarrow==16.1.0
# pyarrow-hotfix==0.6
# pyasn1==0.6.1
# pyasn1_modules==0.4.1
# pycocotools==2.0.8
# pycountry==24.6.1
# pycparser==2.22
# pydantic==2.9.2
# pydantic_core==2.23.4
# pydata-google-auth==1.8.2
# pydot==3.0.2
# pydot-ng==2.0.0
# pydotplus==2.0.2
# PyDrive==1.3.1
# PyDrive2==1.20.0
# pyerfa==2.0.1.4
# pygame==2.6.1
# Pygments==2.18.0
# PyGObject==3.42.1
# PyJWT==2.9.0
# pymc==5.16.2
# pymystem3==0.2.0
# pynvjitlink-cu12==0.3.0
# pyogrio==0.10.0
# PyOpenGL==3.1.7
# pyOpenSSL==24.2.1
# pyparsing==3.1.4
# pyperclip==1.9.0
# pyproj==3.7.0
# pyproject_hooks==1.2.0
# pyshp==2.3.1
# PySocks==1.7.1
# pytensor==2.25.5
# pytest==7.4.4
# python-apt==0.0.0
# python-box==7.2.0
# python-dateutil==2.8.2
# python-dotenv==1.0.1
# python-louvain==0.16
# python-slugify==8.0.4
# python-utils==3.9.0
# pytz==2024.2
# pyviz_comms==3.0.3
# PyYAML==6.0.2
# pyzmq==24.0.1
# qdldl==0.1.7.post4
# ratelim==0.1.6
# ray==2.37.0
# referencing==0.35.1
# regex==2024.9.11
# requests==2.32.3
# requests-oauthlib==1.3.1
# requirements-parser==0.9.0
# rich==13.9.2
# rmm-cu12==24.6.0
# rpds-py==0.20.0
# rpy2==3.4.2
# rsa==4.9
# s3transfer==0.10.3
# safetensors==0.4.5
# scikit-image==0.24.0
# scikit-learn==1.5.2
# scipy==1.13.1
# scooby==0.10.0
# scs==3.2.7
# seaborn==0.13.2
# SecretStorage==3.3.1
# Send2Trash==1.8.3
# sentencepiece==0.2.0
# shapely==2.0.6
# shellingham==1.5.4
# simple-parsing==0.1.6
# six==1.16.0
# sklearn-pandas==2.2.0
# smart-open==7.0.5
# sniffio==1.3.1
# snowballstemmer==2.2.0
# sortedcontainers==2.4.0
# soundfile==0.12.1
# soupsieve==2.6
# soxr==0.5.0.post1
# spacy==3.7.5
# spacy-legacy==3.0.12
# spacy-loggers==1.0.5
# Sphinx==5.0.2
# sphinxcontrib-applehelp==2.0.0
# sphinxcontrib-devhelp==2.0.0
# sphinxcontrib-htmlhelp==2.1.0
# sphinxcontrib-jsmath==1.0.1
# sphinxcontrib-qthelp==2.0.0
# sphinxcontrib-serializinghtml==2.0.0
# SQLAlchemy==2.0.35
# sqlglot==25.1.0
# sqlparse==0.5.1
# srsly==2.4.8
# stanio==0.5.1
# starlette==0.38.6
# statsmodels==0.14.4
# StrEnum==0.4.15
# sympy==1.13.3
# tables==3.8.0
# tabulate==0.9.0
# tbb==2021.13.1
# tblib==3.0.0
# tenacity==9.0.0
# tensorboard==2.17.0
# tensorboard-data-server==0.7.2
# tensorflow==2.17.0
# tensorflow-datasets==4.9.6
# tensorflow-hub==0.16.1
# tensorflow-io-gcs-filesystem==0.37.1
# tensorflow-metadata==1.16.0
# tensorflow-probability==0.24.0
# tensorstore==0.1.66
# termcolor==2.5.0
# terminado==0.18.1
# text-unidecode==1.3
# textblob==0.17.1
# tf-slim==1.1.0
# tf_keras==2.17.0
# thinc==8.2.5
# threadpoolctl==3.5.0
# tifffile==2024.9.20
# tiktoken==0.7.0
# tinycss2==1.3.0
# tokenizers==0.20.1
# toml==0.10.2
# tomli==2.0.2
# toolz==0.12.1
# torch==2.4.0
# torchaudio @ https://download.pytorch.org/whl/cu121_full/torchaudio-2.4.1%2Bcu121-cp310-cp310-linux_x86_64.whl#sha256=da8c87c80a1c1376a48dc33eef30b03bbdf1df25a05bd2b1c620b8811c7b19be
# torchsummary==1.5.1
# torchvision==0.19.0
# tornado==6.3.3
# tqdm==4.66.5
# traitlets==5.7.1
# traittypes==0.2.1
# transformers==4.45.2
# triton==3.0.0
# tweepy==4.14.0
# typeguard==4.3.0
# typer==0.12.5
# types-pytz==2024.2.0.20241003
# types-setuptools==75.1.0.20240917
# typing_extensions==4.12.2
# tzdata==2024.2
# tzlocal==5.2
# uc-micro-py==1.0.3
# uritemplate==4.1.1
# urllib3==2.2.3
# uvicorn==0.31.1
# uvloop==0.20.0
# vega-datasets==0.9.0
# vllm==0.6.2
# wadllib==1.3.6
# wasabi==1.1.3
# watchfiles==0.24.0
# wcwidth==0.2.13
# weasel==0.4.1
# webcolors==24.8.0
# webencodings==0.5.1
# websocket-client==1.8.0
# websockets==13.1
# Werkzeug==3.0.4
# widgetsnbextension==3.6.9
# wordcloud==1.9.3
# wrapt==1.16.0
# xarray==2024.9.0
# xarray-einstats==0.8.0
# xformers==0.0.27.post2
# xgboost==2.1.1
# xlrd==2.0.1
# xxhash==3.5.0
# xyzservices==2024.9.0
# yarl==1.13.1
# yellowbrick==1.5
# yfinance==0.2.44
# zict==3.0.0
# zipp==3.20.2
# zstandard==0.23.0

In [ ]:
# model.cpu()

In [ ]:
# del model

In [ ]:
# torch.cuda.empty_cache()

## MODELLO QUANTIZZATO CON GPTQ (RICHIEDE UNA GPU CON PIU' DI 15 GB DI VRAM)

In [ ]:
# !pip install --upgrade transformers auto-gptq accelerate datasets
# !python -m pip install git+https://github.com/huggingface/optimum.git

In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from optimum.gptq import GPTQQuantizer
# import torch
# model_path = 'Qwen/Qwen1.5-7B'
# w = 4 #quantization to 4-bit. Change to 2, 3, or 8 to quantize with another precision

# # quant_path = 'Qwen1.5-7B-gptq-'+str(w)+'bit'

# # Load model and tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
# model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, device_map="auto")
# quantizer = GPTQQuantizer(bits=w, dataset="c4", model_seqlen = 2048)
# quantized_model = quantizer.quantize_model(model, tokenizer)

# # quantized_model.save_pretrained("./"+quant_path, safetensors=True)
# # tokenizer.save_pretrained("./"+quant_path)

In [ ]:
# model.cpu()

In [ ]:
# del model

In [ ]:
# torch.cuda.empty_cache()

## MODELLO QUANTIZZATO CON AUTOAWQ (RICHIEDE UNA GPU CON PIU' DI 15 GB DI VRAM)

In [ ]:
# !pip install --upgrade torch
# !pip install --upgrade transformers autoawq optimum accelerate

In [ ]:
# from awq import AutoAWQForCausalLM
# from transformers import AutoTokenizer

# model_path = 'Qwen/Qwen1.5-7B'
# # quant_path = 'Qwen1.5-7B-awq-4bit'
# quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }

# # Load model and tokenizer
# model = AutoAWQForCausalLM.from_pretrained(model_path, safetensors=True)
# tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)

# # Quantize
# model.quantize(tokenizer, quant_config=quant_config)

# # Save quantized model with safetensors
# # model.save_quantized("./"+quant_path, safetensors=True)
# # tokenizer.save_pretrained("./"+quant_path)


## MODELLO SUPERQUANTIZZATO CON UNSLOTH

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install unsloth
# Get latest Unsloth
!pip install --upgrade --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/Qwen2-0.5b-bnb-4bit",           # Qwen2 2x faster!
    "unsloth/Qwen2-1.5b-bnb-4bit",
    "unsloth/Qwen2-7b-bnb-4bit",
    "unsloth/Qwen2-72b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # Try more models at https://huggingface.co/unsloth!

model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/Qwen2-7B", # Reminder we support ANY Hugging Face model!
    model_name = "unsloth/Qwen2.5-7B-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)


## Run the model

In [ ]:
data_sel = data #.iloc[:3]

In [ ]:
data_sel = update_data(data_sel)

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
# @title
# results = []

# while 1:

#   data_sel = update_data(data_sel) # AGGIORNO IL DATAFRAME CON I DATI DA PROCESSARE A OGNI STEP

#   if data_sel.shape[0]==0:
#     break


#   for index, row in data_sel.iterrows():

#     cnt = index

#     text_id = row['text_id']
#     text = row["text"].replace('\n','').replace('{','(').replace('}',')')
#     sdg_true = row["sdg_true"]

#     file = open(log_filename, 'a+', encoding='utf-8')

#     print("\n\n##################################################")
#     file.write("\n\n##################################################\n")

#     print("TEXT:", row["text"].replace('\n',''),'\n\n')
#     file.write(f"TEXT: " + row['text'].replace('\n','') + "\n\n\n")

#     template = """Given the following input text, between triple quotes, with its associated classification label with respect to one of the Sustainable Development Goals (SDGs), further classify the text with respect to the three labels defined below:

#     - "positive": the text implies or explicitly states that what is affirmed or described leads to a significant advancement in favor of the goal indicated by the SDG under which it was previously classified.

#     - "negative": the text implies or explicitly states that what is affirmed or described leads to a significant advancement against the goal indicated by the SDG under which it was previously classified.

#     - "neutral": the text does not imply or explicitly state that what is affirmed or described leads to a significant advancement either in favor or against the goal indicated by the SDG under which it was previously classified.

#   Return the result in JSON format with 4 keys:

#     - "label": The assigned label for the input text [this key/value pair is mandatory].

#     - "explanation_1": A description of the reasoning behind the classification decision [this key/value pair is mandatory].

#     - "explanation_2": Additional description of the reasoning behind the classification decision [this key/value pair is optional].

#     - "explanation_3": Another additional description of the reasoning behind the classification decision [this key/value pair is optional].


#   The input text with its label are:
#   '''
#   INPUT TEXT: \""""+text+"""\"
#   SDG_CLASSIFICATION: "SDG-"""+str(sdg_true)+"""\"
#   OUTPUT:
#   '''
#   """

#     # template = "Ciao come stai?"

#     #print(template)

#     llm.generate([template], sampling_params)

#     generated_text = output[0].outputs[0].text

#     result_extended = (cnt, text_id, sdg_true, generated_text)

#     print(result_extended)
#     file.write(str(result_extended)+'\n')

#     with open(output_data_path+f'/sdg_classification_{text_id}.pkl', 'wb') as f:
#         pickle.dump(result_extended, f)

#     add_record(table, str(text_id)) # AGGIUNGE L'ID DEL RECORD PROCESSATO NELLA TABELLA DI DYNAMODB, IN MODO CHE GLI ALTRI WORKER NON LO RIPROCESSINO

#     file.close()

#     break # PER FORZARE L'USCITA DAL CICLO INTERNO E PERMETTERE L'AGGIORNAMENTO DEL DATAFRAME

In [ ]:
results = []

while 1:

  data_sel = update_data(data_sel) # AGGIORNO IL DATAFRAME CON I DATI DA PROCESSARE A OGNI STEP

  if data_sel.shape[0]==0:
    break


  for index, row in data_sel.iterrows():

    cnt = index

    text_id = row['text_id']
    text = row["text"].replace('\n','').replace('{','(').replace('}',')')
    sdg_true = row["sdg_true"]

    file = open(log_filename, 'a+', encoding='utf-8')

    print("\n\n##################################################")
    file.write("\n\n##################################################\n")

    print("TEXT:", row["text"].replace('\n',''),'\n\n')
    file.write(f"TEXT: " + row['text'].replace('\n','') + "\n\n\n")

    template = """Given the following input text, between triple quotes, with its associated classification label with respect to one of the Sustainable Development Goals (SDGs), further classify the text with respect to the three labels defined below:

  - "positive": the text implies or explicitly states that what is affirmed or described leads to a significant advancement in favor of the goal indicated by the SDG under which it was previously classified.

  - "negative": the text implies or explicitly states that what is affirmed or described leads to a significant advancement against the goal indicated by the SDG under which it was previously classified.

  - "neutral": the text does not imply or explicitly state that what is affirmed or described leads to a significant advancement either in favor or against the goal indicated by the SDG under which it was previously classified.

Return the result in JSON format with 4 keys:

  - "label": The assigned label for the input text [this key/value pair is mandatory].

  - "explanation_1": A description of the reasoning behind the classification decision [this key/value pair is mandatory].

  - "explanation_2": Additional description of the reasoning behind the classification decision [this key/value pair is optional].

  - "explanation_3": Another additional description of the reasoning behind the classification decision [this key/value pair is optional].
"""

    input = """INPUT TEXT: \""""+text+"""\"
    SDG_CLASSIFICATION: "SDG-"""+str(sdg_true)+"""\"
    """

    FastLanguageModel.for_inference(model) # Unsloth has 2x faster inference!
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            template, #"Continue the fibonnaci sequence.", # instruction
            input, #"1, 1, 2, 3, 5, 8", # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    # prompt_length = len(alpaca_prompt)+inputs['input_ids'].shape[1]

    outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)
    generated_text = tokenizer.batch_decode(outputs)[0]

    result_extended = (cnt, text_id, sdg_true, generated_text)

    print(result_extended)
    file.write(str(result_extended)+'\n')

    with open(output_data_path+f'/sdg_classification_{text_id}.pkl', 'wb') as f:
        pickle.dump(result_extended, f)

    add_record(table, str(text_id)) # AGGIUNGE L'ID DEL RECORD PROCESSATO NELLA TABELLA DI DYNAMODB, IN MODO CHE GLI ALTRI WORKER NON LO RIPROCESSINO

    file.close()

    break # PER FORZARE L'USCITA DAL CICLO INTERNO E PERMETTERE L'AGGIORNAMENTO DEL DATAFRAME

Amount of processed files:  1054


##################################################
TEXT: This allows an analysis of the population of founders seeking financing in a way that has not previously been possible, and a direct investigation of whether and why gender appeals to be an important determinant of fundraising success. Examples include (but are not restricted to) Alexy et al. ( First, Crunchbase cannot avoid selection, insofar as start-ups initially may also seek other platforms or ways to receive financing. This does not necessarily pose a problem for investigating the gender gap, as long as there are no grounds to think that this self-selection is related to gender. 


(1994, 2656, 5, 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGiven the following input text, between triple quotes, with its associated classification label with respect to one o

ClientError: An error occurred (ExpiredTokenException) when calling the Scan operation: The security token included in the request is expired